In [1]:
using JuliaFEM: Material, set_material!, get_element_set, ElasticityProblem,
DirichletProblem, LoadCase, DisplacementBC, ForceBC, DirichletProblem,
add_boundary_condition!, add_loadcase!

In [2]:

# Reading nodes, elements and sets from input file
model = open(JuliaFEM.parse_abaqus, "/home/olli/.julia/v0.4/JuliaFEM/geometry/piston/piston_8789_P1.inp")

# Creating material
# It is possible to create material data with one function
steel = Material("steel",
    ("youngs modulus" => 210.0e3,
     "poissons ratio" => 0.3,
     "other values" => 100e3),
)

# Or by adding row by row
steel2 = Material("Steel2")
steel2["youngs modulus"] = 210.0e3
steel2["poissons ratio"] = 10.0

# Setting material to element set
set_material!(model, steel, "PISTON")



INFO: Parsing nodes
INFO: Parsing elements. Type: JuliaFEM.Tet4
INFO: Parsing elements. Type: JuliaFEM.Seg3
INFO: Creating elset BC1
INFO: Creating elset BC2


JuliaFEM.Material{ASCIIString,Float64}("steel",Dict("poissons ratio"=>0.3,"youngs modulus"=>210000.0,"other values"=>100000.0))

INFO: Creating elset BC3


In [3]:
# Fetchig set from model
piston = get_element_set(model, "PISTON")

# Creating problems:
# Elasticity 
elasticity_problem = ElasticityProblem()

# Boundary conditions
BC1 = DisplacementBC("displacement u1", 0.0, "BC2")
F1 = ForceBC("displacement traction force", 1000, "BC1")


JuliaFEM.NeumannBC{ASCIIString}("displacement traction force",1000,"BC1")

In [4]:

# Possible ways to create a loadcase, whihc holds all data for calculation
load4 = LoadCase(elasticity_problem)
load = LoadCase(elasticity_problem, [BC1])
load2 = LoadCase(elasticity_problem, [BC1, F1])
load3 = LoadCase(elasticity_problem, BC1)

# Couple of ways to add boundary conditions to loadcase
add_boundary_condition!(load, BC1)
add_boundary_condition!(load2, F1)
add_boundary_condition!(load3, BC1)
add_boundary_condition!(load4, [F1, BC1])
add_boundary_condition!(load, [F1, BC1])
add_boundary_condition!(load4, [F1, BC1])


2-element Array{Array{Union{JuliaFEM.DirichletBC{S<:AbstractString},JuliaFEM.NeumannBC{S<:AbstractString}},1},1}:
 Union{JuliaFEM.DirichletBC{S<:AbstractString},JuliaFEM.NeumannBC{S<:AbstractString}}[JuliaFEM.NeumannBC{ASCIIString}("displacement traction force",1000,"BC1"),JuliaFEM.DirichletBC{ASCIIString}("displacement u1",0.0,"BC2"),JuliaFEM.NeumannBC{ASCIIString}("displacement traction force",1000,"BC1"),JuliaFEM.DirichletBC{ASCIIString}("displacement u1",0.0,"BC2")]
 Union{JuliaFEM.DirichletBC{S<:AbstractString},JuliaFEM.NeumannBC{S<:AbstractString}}[JuliaFEM.NeumannBC{ASCIIString}("displacement traction force",1000,"BC1"),JuliaFEM.DirichletBC{ASCIIString}("displacement u1",0.0,"BC2"),JuliaFEM.NeumannBC{ASCIIString}("displacement traction force",1000,"BC1"),JuliaFEM.DirichletBC{ASCIIString}("displacement u1",0.0,"BC2")]

In [6]:

# Adding load case to model, various ways...
add_loadcase!(model, load)
add_loadcase!(model, [load, load2, load3])
0

0

In [ ]:
# Select solver
solver = DirectSolver()

solve!(model, solver)